
[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.


https://www.youtube.com/watch?v=NRVaRXDoI3g

https://colab.research.google.com/drive/1Vvju5kOyBsDr7RX_YAvp6ZsSOoSMjhKD?usp=sharing

https://huggingface.co/dfurman/falcon-7b-openassistant-peft/blob/main/finetune_falcon7b_oasst1_with_bnb_peft.ipynb

```
Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA
https://huggingface.co/blog/4bit-transformers-bitsandbytes

Tim Dettmers Hugging Face Profile - https://huggingface.co/timdettmers

QLORA: Efficient Finetuning of Quantized LLMs Paper - https://arxiv.org/pdf/2305.14314.pdf
```

이번 실습에 사용할 GPU는 Colab에서 무료로 제공하는 T4 GPU입니다.

In [ ]:
!nvidia-smi

Sun Sep 17 13:31:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Colab 무료 GPU인 T4로 Polyglot-Ko 12.8B 파인 튜닝 가능  
- seq_length를 1024로 하고 Batch 사이즈 2 가능. T4 (16G)  
- seq_length를 1024로 하고 Batch 사이즈 8 가능. A100 (40G) : 유료 구독 Colab GPU

12.8B가 RAM을 많이 먹어서 가급적이면 Colab 유료 버전을 결제하시고 '고용량 RAM'을 체크 후에 실행하시는 것이 좋습니다.

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

https://huggingface.co/datasets/beomi/KoAlpaca-v1.1a

In [ ]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [ ]:
data['train']

Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 21155
})

In [ ]:
data['train'][0]

{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
 'output': '양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
 'url': 'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=1116&docId=55320268'}

기존 데이터셋의 instruction과 output을 이용하여 text라는 새로운 형식을 만듭니다.

In [ ]:
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
data['train']['text'][0]

'### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n### 답변: 양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.<|endoftext|>'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21155
    })
})

In [ ]:
data['train']

Dataset({
    features: ['instruction', 'output', 'url', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 21155
})

In [ ]:
print('첫번째 샘플의 정수 인코딩 :', data['train']['input_ids'][0])
print('첫번째 샘플의 정수 인코딩 복원 :', tokenizer.decode(data['train']['input_ids'][0]))

첫번째 샘플의 정수 인코딩 : [6, 6, 6, 2438, 29, 11535, 272, 1382, 6587, 6548, 11260, 34, 1273, 543, 425, 9751, 4408, 11260, 34, 202, 202, 6, 6, 6, 4253, 29, 11535, 272, 9964, 270, 1577, 6587, 285, 8260, 1649, 1441, 17, 543, 425, 9751, 6587, 285, 4408, 1649, 1441, 17, 224, 202, 202, 9600, 285, 6548, 285, 4878, 274, 879, 3716, 20739, 272, 669, 270, 2016, 2670, 307, 2438, 274, 741, 1865, 276, 1044, 283, 327, 276, 388, 1441, 17, 11535, 272, 9964, 270, 1577, 8260, 1649, 1441, 17, 543, 425, 9751, 1108, 2438, 359, 4253, 363, 2958, 551, 388, 359, 2192, 4408, 1649, 1441, 17, 3322, 15, 11535, 272, 6587, 285, 8260, 1649, 270, 740, 283, 15, 543, 425, 9751, 6587, 285, 4408, 1649, 1441, 17, 202, 202, 26205, 272, 4253, 29, 15583, 8260, 309, 11228, 19320, 276, 365, 327, 4389, 34, 224, 202, 202, 283, 11946, 8260, 309, 19126, 339, 11228, 19320, 276, 365, 327, 827, 17, 1146, 8260, 1708, 380, 769, 346, 15, 9964, 15, 1235, 15, 4408, 617, 1392, 6548, 293, 19126, 339, 1495, 605, 15596, 2802, 17, 2466, 15, 803, 363, 272

In [ ]:
print('첫번째 샘플의 세그먼트 인코딩 :', data['train']['token_type_ids'][0])
print('첫번째 샘플의 어텐션 마스크 :',data['train']['attention_mask'][0])

첫번째 샘플의 세그먼트 인코딩 : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
첫번째 샘플의 어텐션 마스크 : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


무료 Colab에서 사용 가능한 T4 GPU를 사용한 예시입니다.

In [ ]:
!nvidia-smi

Sun Sep 17 14:39:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    28W /  70W |   9097MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
tokenizer.eos_token

'<|endoftext|>'

In [ ]:
tokenizer.pad_token

'<|endoftext|>'

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.400200
20,2.248400
30,2.011900
40,2.066100
50,1.897600
60,2.002900
70,1.885500
80,1.912300
90,1.852200
100,1.950400


TrainOutput(global_step=500, training_loss=1.8747712860107422, metrics={'train_runtime': 2518.1747, 'train_samples_per_second': 0.397, 'train_steps_per_second': 0.199, 'total_flos': 2.156311716225024e+16, 'train_loss': 1.8747712860107422, 'epoch': 0.05})

500 step 학습하였으며 0.05 에포크라고 나옵니다. 다시 말해서 1에포크도 못돌렸습니다.

RunPods이라는 플랫폼을 사용하면, 비용은 A100 80GB 1장을 1시간 사용하는데, 2400원 정도 드는 플랫폼이며, AWS나 Colab보다도 훨씬 저렴합니다. 배치 크기를 늘리고 더 빠른 학습을 하고 싶다면 검토해보세요.

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1349: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34,  224,  202, 4588,
         4770,  272, 1382, 5674,   34, 5000, 9357,  272]])

In [ ]:
def generate(x):
    generated_sentence = model.generate(**tokenizer(f"### 질문: {x}\n\n### 답변:", return_tensors='pt', return_token_type_ids=False),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(generated_sentence[0]))

Colab 무료 GPU T4 기준:  
- 답변을 얻기 위해 걸린 소요 시간: 약 1분

In [ ]:
generate('순환 신경망에 대해서 알려주세요')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 순환 신경망에 대해서 알려주세요

### 답변: 순환 신경망은 기본적으로 신경망의 확장 버전입니다. 신경망의 각 층은 독립적으로 존재하는 반면, 순환 신경망은 각 층의 연결이 전체 연결을 결정합니다. 따라서, 다른 신경망의 입력 및 입력 데이터에 의해 다음 층의 출력이 영향을 받습니다. 이러한 방식으로 학습 프로세스는 순환적으로 일어나며, 이 순환 신경망은 심층 신경망이 갖는 특징과 유사한 개념을 갖습니다. 따라서, 순환 신경망은 다른 신경망에 비해 훨씬 많은 연결을 수행해야 하기 때문에 처리 시간이 오래 걸리며, 전체 모델이 완성되기 전에 학습 데이터가 부족한 경우 성능이 좋지 않습니다. 또한, 순환 신경망은 층별 연결 관계를 사용하여 새로운 입력과 출력값 사이의 연결을 반복적으로 조정하기 때문에, 예측 오차가 큰 단점이 있습니다. 따라서, 전체적인 성능을 향상시킬 수 있는 기술은 아직 개발되지 않았습니다. 이러한 순환 신경망의 단점을 극복하기 위해서, 합성곱 계층의 연결 관계를 사용하는 방법, 학습 단계에서 학습을 반복할 위치를 결정하는 방법 등 다양한 접근법이 제안되어 연구되고 있


In [ ]:
generate('허리가 아플 때는 어떻게 하는 게 좋을까요?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 허리가 아플 때는 어떻게 하는 게 좋을까요?

### 답변: 허리가 아플 때는 우선 휴식을 가지는 것이 가장 중요합니다. 충분한 휴식 후에도 증상이 완화되지 않는다면 바로 병원에 가보는 것이 현명합니다. 허리가 아프다고 해서 모두 디스크일 가능성은 굉장히 낮으며, 근육통과 연관이 있을 가능성이 높습니다. 따라서 아픈 부위를 마사지 해주고 따뜻한 핫팩을 허리에 가져다대는 것이 좋습니다. 그러나 이러한 방법은 일시적인 효과만 있기 때문에 근본적인 해결 방법이 가장 중요합니다. 따라서 허리통증을 개선하기 위해서는 운동을 통해 코어 근육과 유연성을 강화시키는 것이 가장 좋습니다. 허리 스트레칭이나 플랭크, 코어운동을 통해 허리 근육을 강화시켜 주어야 합니다. 또한 수면 자세를 고쳐주는 것도 허리통증을 완화하는데 중요합니다. 따라서 올바른 자세로 자는 것이 중요하며, 다리를 펴고 자는 것은 허리디스크 예방을 위한 필수적인 습관입니다. 이외에도 여러가지 허리 통증을 완화시키기 위한 노력들이 있지만, 가장 중요한 것은 허리에 무리를 주지 않고, 유연성을 길러주며, 충분한 휴식을 취하는 것입니다. 허리디스크를 방치


In [ ]:
generate('주식 시장에서 안정적으로 수익을 얻기 위한 방법을 추천해주십쇼')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1539: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 주식 시장에서 안정적으로 수익을 얻기 위한 방법을 추천해주십쇼

### 답변: 현재 국내 주식 시장은 약 1000조 원에 달하는 세계 3위 규모의 시장입니다. 기관투자자의 비중도 약 45%에 이를 정도로 주식 시장에서는 중 · 장년 층의 자금이 유입되고 있습니다. 주식 시장에 안정적으로 수익을 얻기 위해서는 '장기 가치 투자'를 하셔야 합니다. 장기투자는 10년 이상의 투자 기간을 의미하는 것이 아닌, '좋은 기업의 주식을 장기간 보유한다'는 것을 의미합니다. 이는 가치 투자로도 불리며, 워렌 버핏의 투자 전략으로 널리 알려져있습니다. 

좋은 기업을 찾기 위해, 기업의 재무제표와 기술 산업 동향 등 다양한 정보와 자료를 수집하셔야 합니다. 이를 통해 자신이 투자할 기업을 선택하고, 해당 기업의 주식을 꾸준히 보유하고 있으면 장기 투자 전략을 실천할 수 있습니다. 
특히 기관투자자의 특성상, 장기 투자에 유리한 우량주나, 산업 경쟁력이 뛰어난 기업을 선호하기 때문에 이러한 종목에 주식 시장의 수익률을 극대화시킬 수 있습니다. 이러한 전략을 실천하기 위해서는 해당 기업의 제품을 잘 알 필요도 있습니다. 예


In [ ]:
generate('한국에서 가장 유명한 아이돌 누구야')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 한국에서 가장 유명한 아이돌 누구야

### 답변: 방탄소년단, 슈퍼주니어, 동방신기, 블랙핑크, EXO, 샤이니, 위너, 빅스 등의 다수의 아이돌이 한국에서 가장 인기 있는 아이돌입니다. 이들은 모두 한국어 가사와 노래, 외모, 춤과 연기력 등 여러 방면에서 우수한 점을 가지고 있습니다. 이 외에도 비(정지훈), 세븐, 보아 등도 유명하며, 남자 그룹으로는 빅뱅, GOT7, BEAST, 뉴이스트W, 아스트로, B1A4, 방탄소년단 등이 있습니다. 여자 그룹으로는 트와이스, 걸스데이, 레드벨벳, 러블리즈, 마마무, 여자친구, 소녀시대, AOA, 에이핑크, 소녀시대, F(x)(에프엑스), 나인뮤지스, 우주소녀, 오마이걸, 프로미스나인, 다이아, 구구단, 모모랜드, 블랙핑크, 카라, 투애니원, 2NE1, 미쓰에이 등이 있습니다. 참고로 위키백과는 "대한민국에서는 "아이돌'이란 용어가 처음으로 널리 쓰이기 시작한 1990년대 중-후반부에 데뷔한 "이른바 1세대 아이돌"이 가장 인기를 끌었고, 2000년대 초반에


In [ ]:
generate('수험생인데 공부 잘 하는 법 알려줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 수험생인데 공부 잘 하는 법 알려줘

### 답변: 수능 100일 전부터 시험 당일까지를 수능일 D-100일 전략으로 시작합니다. 이 전략을 통해 수험생들은 계획적으로, 체계적으로 시간을 관리하며 공부할 수 있습니다. 또한 수능일이 가까워 올수록 시간 여유가 줄어들어 벼락치기식 공부가 되기 쉬운데, 화성시와 같이 수험생들을 위해 다양한 이벤트를 하는 도시도 있으므로, 자신이 사는 지역에서 개최하는 박람회나 이벤트에 참석하는 것도 도움이 됩니다. 또한 여름방학 때는 기출문제를 집중적으로 분석하고, 모의고사와 오답노트을 통해 취약점을 발견하여 보완하는 것이 좋습니다. 모의고사를 풀 때는 자신의 실력보다 20% 아래의 문제부터 풀고, 모르는 부분을 확인하여 공부하는 것이 좋습니다. 마지막 D-100일 부터는 새로운 부분을 공부하기보다는 기출문제나 정리한 요약집을 보는 것이 좋습니다. 시험 당일에는 전날부터 소화가 잘 되는 음식 위주로 먹거나 시험시간에는 너무 어렵게 생각하지 않고 아는 문제부터 푸는 것이 좋습니다. 문제를 풀 때는 침착하게 풀고, 헷갈리는 문제나 잘 모르는


In [ ]:
generate('이순신 장군님에 대해서 설명해줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 이순신 장군님에 대해서 설명해줘

### 답변: 조선시대 무관으로서 임진왜란 당시 수많은 왜군과 맞붙어 이기고 조선을 지킨 명장입니다. 임진왜란 당시에는 옥포해전, 한산해전, 명량해전에서 승리하면서도 노량해전에서 유탄에 맞아 사망하는 순간까지도 왜군을 쳐부수는 승리를 이끌어 냈습니다. 장군의 본관은 덕수, 자는 여해, 시호는 충무입니다. 장군은 무과에 급제하여 1576년 (선조 9)에 외인(外人), 종9품인 건원보 권관이 되어 함경도 동구비보의 외곽 방어 및 순찰 책임을 맡았습니다. 1579년 (선조 12년) 훈련원 봉사, 임진왜란이 발발했을 때에 발포 수군 만호, 전라 좌수영의 군관(종6품), 옥포해전 때에는 중위장(중위)을 거쳐 가선대부(종2품)로 특진하여 전라좌도 수군절도사에 오른 후 임진왜란 중 크고 작은 13개 전투에서 승전을 거두며 큰 공을 세웠습니다. 

조선 중기 조선의 명장으로 활약한 이순신 장군님도 충무공 시호를 받았습니다. �
이순신 장군


In [ ]:
generate('여자 친구 만나는 법 알려줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 여자 친구 만나는 법 알려줘

### 답변: 소개팅은 본인이 적극적으로 하거나 친구들의 도움을 받는 것도 좋습니다. 학교 내에서 잘 아는 사람을 찾는 것도 좋고, 동아리나 모임에 참석해보는 것도 방법이 될 수 있습니다. 주변에서 소개팅을 제안하는 것도 있습니다. 이는 학교 내에서 이루어지는 경우가 많으며, 학교 밖에서 이루어지는 경우가 많습니다. 소개받은 사람과 잘 만나기 위해선, 그 사람의 특성을 잘 파악해서 적극적으로 다가가는 것도 좋은 방법입니다. 
소개팅의 경우는 다음과 같은 질문을 통해 답을 얻을 수 있습니다.  

1. 나이 차이가 몇 살이 좋은가? 
2. 종교가 있는가? 
3. 좋아하는 요리는 무엇인가? 4. 좋아하는 노래는 무엇인가? 
5. 가장 자주 사용하는 폰트는 무엇인가? 
6. 최근에 본 영화는 무엇인가? 
7. 여행 가고 싶은 곳이 있는가? 
8. 좋아하는 드라마는 무엇인가? 
9. 지금까지 가본 최고의 해외여행은 어디인가? 
10
